# Phase 1 

### Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [5]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])


*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

In [6]:
# need to get the file names in the bug report an source code to be the same
display(all_projects_bugreports.iloc[0].fix)
display(all_projects_source_codes.filename)


array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

0     \gitrepo\src\java\org\apache\commons\collectio...
1     \gitrepo\src\java\org\apache\commons\collectio...
2     \gitrepo\src\java\org\apache\commons\collectio...
3     \gitrepo\src\java\org\apache\commons\collectio...
4     \gitrepo\src\java\org\apache\commons\collectio...
                            ...                        
63    \gitrepo\src\main\java\org\wildfly\security\ut...
64    \gitrepo\src\main\java\org\wildfly\security\_p...
65    \gitrepo\src\test\java\org\wildfly\security\ma...
66    \gitrepo\src\test\java\org\wildfly\security\ma...
67    \gitrepo\src\test\java\org\wildfly\security\ss...
Name: filename, Length: 10461, dtype: object

## Removing composite varaibles

In [22]:
import re
#remove next line characters:
def remove_new_lines(text):
#     return str(x).replace('\n', '').replace('*', '').replace('/', '').replace('\\','').replace('\t','')
    text = str(text)
    COMBINE_WHITE_SPACE = re.compile(r"(?a:\s+)")
    text = COMBINE_WHITE_SPACE.sub(' ', text)
    return text.replace('*', '').replace('/', '').replace('\\','')
    
# clean up the various white space and remove some *
def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

# clean up the description and summary, they will both be used for the query
def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|S')
    df.description = df.description.apply(remove_new_lines)
    return df

# need to reformat the source code so it can be compared to the bug reports fix array
def format_sc_filename(x):
  
    x = x.split('apache')
    if len(x) != 2:
        x = x[0].split('springframework')
        if len(x) != 2:
            x = x[0].split('wildfly')
            if len(x) == 2:
                x = 'org.wildfly' + x[1]
        else:
             x = 'org.springframework' + x[1]
    else:
         x = 'org.apache' + x[1]
    if len(x) == 1:
        x = x[0]
   
    x = str(x).replace("\\",".")
    return x

# apply the fixes to the filename 
def clean_sc_filepath(df):
    df.filename = df.filename.apply(format_sc_filename)
    return df



In [8]:


#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df


In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# add the Java key words to the stop words
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
java_operators = ["+", "-", "*", "/", "%", "+=", "-=", "*=", "/=", "++", "--", "==", "!=", "<", ">", "<=", ">=", ".", "[", "]", "(",")", "!", "~","instanceof", "<<", ">>", ">>>", "&", "^", "|", "&&", "||", "?", ":", "^=", "%=", "<<=", ">>=", ">>>=", "&="]
stop = java_keywords + java_operators
STOP_WORDS = ENGLISH_STOP_WORDS.union(stop)
# STOP_WORDS


### Start working on TF-IDF and Cosine similarity calculations

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clean up the unprocessed code column
def clean_source_df(sc_df):
    sc_df = clean_new_lines_source_code(sc_df)
    sc_df = clean_composite_source_code(sc_df)
    sc_df = clean_sc_filepath(sc_df)
    return sc_df

# add the summary and description together and clean the data
def clean_bug_df(sc_df):
    sc_df = clean_new_lines_bug_report(sc_df)
    sc_df = clean_composite_bug_report(sc_df)
    return sc_df

# get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_bug_df(all_projects_bugreports)

display(br_df.iloc[0].description)
br_df["query"] = br_df["summary"] + br_df["description"]
display(br_df.iloc[0].query)

# get a list of the projects 
print(sc_df.project.unique())
projects = sc_df.project.unique()

# group the data frames
sc_grouped_df = sc_df.groupby(sc_df.project)

# example of getting a data frame for a single project
col_df = sc_grouped_df.get_group("COLLECTIONS")
display(col_df)




'Flat3  Map&amp;apos;s  Entry objects will overwrite other  Entry&amp;apos;s values if  Entry.set  Value() is called on one.  It should only overwrite the  Entry at hand.n  I&amp;apos;ve looked at the source, and the case statement incorrectly falls through, rather than returning like it should:n  Flat3  Map.java, lines 646-660:n public  Object set  Value(  Object value) {n if (can  Remove == false) n{n throw new  Illegal  State  Exception(  Abstract  Hashed  Map.  S  E  T  V  A  L  U  E_  I  N  V  A  L  I  D);n }n  Object old = get  Value();n switch (next  Index) n{n case 3: n parent.value3 = value;n case 2:n parent.value2 = value;n case 1:n parent.value1 = value;n }n return old;n }n  With this code, if  I set the value of the third item in the  Entry  Set, then all three values are set to the new value.\'"\'\'\'\'\'\'\''

'Flat3  Map.  Entry.set  Value() overwrites other  Entry valuesFlat3  Map&amp;apos;s  Entry objects will overwrite other  Entry&amp;apos;s values if  Entry.set  Value() is called on one.  It should only overwrite the  Entry at hand.n  I&amp;apos;ve looked at the source, and the case statement incorrectly falls through, rather than returning like it should:n  Flat3  Map.java, lines 646-660:n public  Object set  Value(  Object value) {n if (can  Remove == false) n{n throw new  Illegal  State  Exception(  Abstract  Hashed  Map.  S  E  T  V  A  L  U  E_  I  N  V  A  L  I  D);n }n  Object old = get  Value();n switch (next  Index) n{n case 3: n parent.value3 = value;n case 2:n parent.value2 = value;n case 1:n parent.value1 = value;n }n return old;n }n  With this code, if  I set the value of the third item in the  Entry  Set, then all three values are set to the new value.\'"\'\'\'\'\'\'\''

['COLLECTIONS' 'CONFIGURATION' 'IO' 'LANG' 'DATACMNS' 'DATAMONGO'
 'DATAREST' 'LDAP' 'SEC' 'SOCIALFB' 'SPR' 'ELY']


,filename,unprocessed_code,project
0,org.apache.commons.collections.arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,org.apache.commons.collections.bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,org.apache.commons.collections.bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,org.apache.commons.collections.beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,org.apache.commons.collections.bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
471,org.apache.commons.collections.set.testtransfo...,Licensed to the Apache Software Foundation ...,COLLECTIONS
472,org.apache.commons.collections.set.testtypedse...,Licensed to the Apache Software Foundation ...,COLLECTIONS
473,org.apache.commons.collections.set.testtypedso...,Licensed to the Apache Software Foundation ...,COLLECTIONS
474,org.apache.commons.collections.set.testunmodif...,Licensed to the Apache Software Foundation ...,COLLECTIONS


In [11]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_similarity(source_code, bug):
    cosSim = cosine_similarity(bug, source_code).flatten()
    return cosSim

# fit the vectorizer and transform data
def transform_data(source_code_data, query_data):
    # DO WE SET min_df?  default normalization is l2
    s = [source_code_data]
    q = [query_data]
    vect = TfidfVectorizer(min_df=1).fit(s)
    X = vect.transform(s)
    y = vect.transform(q)
    
    
    similarity = calculate_similarity(X,y)
     
    return X, y, similarity

# remove the stem and stop words
# takes in an array of strings returns an array of strings
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()
    return text

# take in the source code df for a project and return scores
def generate_scores(df):
    
    # get a list of strings from the data frame to be vectorized
    
    # fit a vectorizer to the data
    
    # iterate through the queries(bugs) and rank the similarities


In [ ]:
# similarity vector should be a data frame with bug_report, bug_report_fix, source_code.filename, and the similarity score

# another function that determines if the source_code file name is in the bugreport fix array. 



In [ ]:
print(type(scores['org.apache.commons.collections.arraystack.java']))